In [ ]:
import hashlib
import time
import random

class Block:
    def __init__(self, data, previous_hash):
        self.timestamp = time.time()
        self.data = data
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        return hashlib.sha256(str(self.timestamp).encode() + 
                              str(self.data).encode() + 
                              str(self.previous_hash).encode() + 
                              str(self.nonce).encode()).hexdigest()

    def mine_block(self, difficulty):
        while self.hash[:difficulty] != "0" * difficulty:
            self.nonce += 1
            self.hash = self.calculate_hash()


class Blockchain:
    def __init__(self, difficulty):
        self.chain = [self.create_genesis_block()]
        self.difficulty = difficulty
        self.nodes = [Node(i, random.uniform(0.1, 1.0), self) for i in range(6)]

    def create_genesis_block(self):
        return Block("Genesis Block", "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().hash
        new_block.mine_block(self.difficulty)
        self.chain.append(new_block)

    def is_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]
            if current_block.hash != current_block.calculate_hash():
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True

class Node:
    def __init__(self, id, latency, blockchain):
        self.id = id
        self.latency = latency
        self.blockchain = blockchain
        self.blocks_mined = 0

    def mine_block(self):
        latest_block = self.blockchain.get_latest_block()
        new_block = Block(f"Block {len(self.blockchain.chain)}", latest_block.hash)
        time.sleep(self.latency)  # introduce network latency
        new_block.mine_block(self.blockchain.difficulty)
        self.blockchain.add_block(new_block)
        self.blocks_mined += 1

    def mine_blocks(self, num_blocks):
        for i in range(num_blocks):
            self.mine_block()


if __name__ == '__main__':
    blockchain = Blockchain(2)  #adjust difficulty
    print("Mining genesis block...")
    blockchain.chain[0].mine_block(blockchain.difficulty)
    print("Genesis block mined:", blockchain.chain[0].hash)
    print("\nMining subsequent blocks...")
    for i in range(100):
        nodes = blockchain.nodes
        random.shuffle(nodes)  # shuffle nodes to randomize order
        for node in nodes:
            node.mine_block()
            if blockchain.is_valid():
                print(f"Node {node.id} mined block {len(blockchain.chain) - 1} ")
                break
    print("\nBlockchain is valid:", blockchain.is_valid())
    print("\nComplete blockchain:")
    for block in blockchain.chain:
        print(block.__dict__)
    nodes_sorted = sorted(blockchain.nodes, key=lambda node: node.blocks_mined, reverse=True)
    print("\nBlocks mined by each node (in decreasing order):")
    for node in nodes_sorted:
        print(f"Node {node.id}: {node.blocks_mined} blocks mined ({node.latency:.2f}s)")

    print("\nPercentage of blocks mined by each node:")
    total_blocks = sum([node.blocks_mined for node in blockchain.nodes])
    for node in blockchain.nodes:
    percentage = round(node.blocks_mined / total_blocks * 100, 2)
    print(f"Node {node.id}: {percentage}% of blocks mined, latency={node.latency:.2f}s")
            
